In [1]:
import pandas as pd
from datetime import datetime, timedelta
from gamma import BayesianGaussianMixture, GaussianMixture
from gamma.utils import convert_picks_csv, association, from_seconds
import numpy as np
from sklearn.cluster import DBSCAN 
from datetime import datetime, timedelta
import os
import json
import pickle
from pyproj import Proj
from tqdm import tqdm

In [7]:
data_dir = lambda x: os.path.join("working", x)
station_csv = data_dir("stations1.csv")
picks = data_dir("picks.csv")
catalog_csv = data_dir("catalog_gamma.csv")
picks_csv = data_dir("picks_gamma.csv")


if not os.path.exists("figures"):
    os.makedirs("figures")
figure_dir = lambda x: os.path.join("figures", x)

config = {'center': (-95.3516, 16.3274), 
    'xlim_degree': [-99.5665, -91.1368], 
    'ylim_degree': [13.5593, 19.0956], 
    'degree2km': 106.659, 
    'starttime': datetime(2017, 10, 15, 0, 0), 
    'endtime': datetime(2017, 10, 18, 0, 0)}
## read picks
picks = pd.read_csv(picks)
picks = picks.rename(columns={
    "station_id": "id",
    "phase_time": "timestamp",
    "phase_amp": "amp",
    "phase_type": "type",
    "phase_score": "prob"
})



## read stations
stations = pd.read_csv(station_csv, delimiter="\t")
stations = stations.rename(columns={"station":"id"})
proj = Proj(f"+proj=sterea +lon_0={config['center'][0]} +lat_0={config['center'][1]} +units=km")
stations[["x(km)", "y(km)"]] = stations.apply(lambda x: pd.Series(proj(x["longitude"], x["latitude"])), axis=1)
stations["z(km)"] = stations["elevation(m)"].apply(lambda x: -x/1e3)

### setting GMMA configs
config["use_dbscan"] = True
config["use_amplitude"] = True
config["method"] = "BGMM"  
if config["method"] == "BGMM": ## BayesianGaussianMixture
    config["oversample_factor"] = 4
if config["method"] == "GMM": ## GaussianMixture
    config["oversample_factor"] = 1

# earthquake location
config["vel"] = {"p": 6.0, "s": 6.0 / 1.75}
config["dims"] = ['x(km)', 'y(km)', 'z(km)']
config["x(km)"] = (np.array(config["xlim_degree"])-np.array(config["center"][0]))*config["degree2km"]
config["y(km)"] = (np.array(config["ylim_degree"])-np.array(config["center"][1]))*config["degree2km"]
config["z(km)"] = (0, 20)
config["bfgs_bounds"] = (
    (config["x(km)"][0] - 1, config["x(km)"][1] + 1),  # x
    (config["y(km)"][0] - 1, config["y(km)"][1] + 1),  # y
    (0, config["z(km)"][1] + 1),  # z
    (None, None),  # t
)   

# DBSCAN
config["dbscan_eps"] = 10 #s
config["dbscan_min_samples"] = 3

# filtering
config["min_picks_per_eq"] = min(10, len(stations) // 2)
config["min_p_picks_per_eq"] = 0
config["min_s_picks_per_eq"] = 0
config["max_sigma11"] = 2.0 # s
config["max_sigma22"] = 1.0 # log10(m/s)
config["max_sigma12"] = 1.0 # covariance

## filter picks without amplitude measurements
if config["use_amplitude"]:
    picks = picks[picks["amp"] != -1]

for k, v in config.items():
    print(f"{k}: {v}")

center: (-95.3516, 16.3274)
xlim_degree: [-99.5665, -91.1368]
ylim_degree: [13.5593, 19.0956]
degree2km: 106.659
starttime: 2017-10-15 00:00:00
endtime: 2017-10-18 00:00:00
use_dbscan: True
use_amplitude: True
method: BGMM
oversample_factor: 4
vel: {'p': 6.0, 's': 3.4285714285714284}
dims: ['x(km)', 'y(km)', 'z(km)']
x(km): [-449.5570191  449.5463532]
y(km): [-295.2427779  295.2534438]
z(km): (0, 20)
bfgs_bounds: ((-450.55701910000005, 450.5463532000012), (-296.2427779000001, 296.2534438), (0, 21), (None, None))
dbscan_eps: 10
dbscan_min_samples: 3
min_picks_per_eq: 10
min_p_picks_per_eq: 0
min_s_picks_per_eq: 0
max_sigma11: 2.0
max_sigma22: 1.0
max_sigma12: 1.0


## 2. Associaiton with GaMMA

In [5]:
event_idx0 = 0 ## current earthquake index
assignments = []
catalogs, assignments = association(picks, stations, config, event_idx0, config["method"])
event_idx0 += len(catalogs)

## create catalog
catalogs = pd.DataFrame(catalogs, columns=["time"]+config["dims"]+["magnitude", "sigma_time", "sigma_amp", "cov_time_amp",  "event_index", "gamma_score"])

# where I keep getting the error 
catalogs[["longitude","latitude"]] = catalogs.apply(lambda x: pd.Series(proj(longitude=x["x(km)"], latitude=x["y(km)"], inverse=True)), axis=1)
catalogs["depth(m)"] = catalogs["z(km)"].apply(lambda x: x*1e3)


with open(catalog_csv, 'w') as fp:
    catalogs.to_csv(fp, sep="\t", index=False, 
                    float_format="%.3f",
                    date_format='%Y-%m-%dT%H:%M:%S.%f',
                    columns=["time", "magnitude", "longitude", "latitude", "depth(m)", "sigma_time", "sigma_amp", "cov_time_amp", "event_index", "gamma_score"])
# catalogs = catalogs[['time', 'magnitude', 'longitude', 'latitude', 'depth(m)', 'sigma_time', 'sigma_amp', 'gamma_score']]

## add assignment to picks
assignments = pd.DataFrame(assignments, columns=["pick_index", "event_index", "gamma_score"])
picks = picks1.join(assignments.set_index("pick_index")).fillna(-1).astype({'event_index': int})
with open(picks_csv, 'w') as fp:
    picks.to_csv(fp, sep="\t", index=False, 
                    date_format='%Y-%m-%dT%H:%M:%S.%f',
                    columns=["station_id", "phase_time", "phase_type", "phase_score", "phase_amp", "event_index", "gamma_score"])

Skip 7884 picks: 100%|███████████████████| 9585/9585 [00:00<00:00, 12810.54it/s]


ValueError: Columns must be same length as key